In [9]:
!pip install apscheduler

    100% |████████████████████████████████| 61kB 389kB/s ta 0:00:01
  Running setup.py bdist_wheel for tzlocal ... - \ | done
  Stored in directory: /home/jovyan/.cache/pip/wheels/dc/d6/f0/84194ccbdd3e2551bdb207e1fc1925efd32956c5767328103b
Successfully built tzlocal
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
from pprint import pprint
import datetime
from ebaysdk.exception import ConnectionError
from ebaysdk.finding import Connection
import pandas as pd
from twilio.rest import Client
from datetime import datetime
from pytz import timezone
#import schedule
from apscheduler.schedulers.blocking import BlockingScheduler

In [21]:
#initilize itemid_list or reset if restarted kernel
#itemid_list = [122481295107, 322504020605, 182559071630]

#save itemid_list for next run
#itemid_list

[122481295107, 322504020605, 182559071630, 112398589037]

In [19]:
#request to ebay
api_request = {
            #'keywords': u'niño',
            'keywords': 'Macbook Pro Retina 16gb 15',
            'sortOrder': 'StartTimeNewest'}

In [ ]:
sched = BlockingScheduler()
sec = 20

@sched.scheduled_job('interval', seconds=sec)
def timed_job():
    try:
        api = Connection(appid="GeoffCou-macsearc-PRD-4090b840d-a41c0a0b", config_file=None)
        response = api.execute('findItemsAdvanced', api_request)
    except ConnectionError as e:
        a = 1
        
    #got a response
    r = response.dict()
    
    #create dict to import into pandas
    dates = {}
    for i in range(100):
        dates[i] = {}
        dates[i]['d'] = r['searchResult']['item'][i]['listingInfo']['startTime']
        dates[i]['buy'] = r['searchResult']['item'][i]['listingInfo']['listingType']
        dates[i]['currency'] = r['searchResult']['item'][i]['sellingStatus']['currentPrice']['_currencyId']
        dates[i]['price'] = r['searchResult']['item'][i]['sellingStatus']['currentPrice']['value']
        dates[i]['return'] = r['searchResult']['item'][i]['returnsAccepted']
        dates[i]['condition'] = r['searchResult']['item'][i]['condition']['conditionId']
        dates[i]['url'] = r['searchResult']['item'][i]['viewItemURL']
        dates[i]['categoryid'] = r['searchResult']['item'][i]['primaryCategory']['categoryId']
        dates[i]['shiping'] = r['searchResult']['item'][i]['shippingInfo']['shippingType']
        dates[i]['itemid'] = r['searchResult']['item'][i]['itemId']
    
    #dict df
    d = pd.DataFrame(dates)
    d = d.T

    #convert time
    d['d'] = d['d'].str.replace('T',' ')
    d['d'] = d['d'].str.replace('Z','')
    #d['d'] = d['d'].str[:-4]
    d['d'] = pd.to_datetime(d['d'])

    #to numeric
    for col in d.columns:
        if col not in ['time','d']:
            try:
                d[col] = pd.to_numeric(d[col])
            except:
                ValueError
    
    #dont cut the 
    pd.set_option('display.max_colwidth', -1)
                
    #time fudging
    #d['d'] = d['d'].apply(timezone('US/Eastern').localize)
    #d['diff'] = d['d'] - datetime.now(timezone('US/Eastern'))        

    accountSID = 'ACba8d171da71289807d6aa640b2e12eb8'
    authToken = 'fbe13c9f5a79c3b4922530d0e93b03b6'
    client = Client(accountSID, authToken)
    myTwilioNumber = '+13399707227'
    myCellPhone = '+13392239032'
    
    #conditions of url message
    #minutes = 600 #doesnt' matter anymore now that i sorted the ebay query by time
    price = 750
    mask = ((d['buy'] == 'FixedPrice') 
            & (d['categoryid'] == 111422) 
            & (d['price'] <= price))

    #initialize list
    #itemid_list = []

    #loop over dictionary
    url = ''
    for i in range(len(d[mask])):
        if d[mask]['itemid'].iloc[i] not in itemid_list:
            itemid_list.append(d[mask]['itemid'].iloc[i])
            #print(i)
            url = str(d[mask]['url'].iloc[i]) + ' AND ' + url 
            #print(url)
            message = client.messages.create(body=str(url),from_=myTwilioNumber, to=myCellPhone)
    #print(d[mask])
    #print every run
    print('last ran: ' + str(datetime.now(timezone('US/Eastern'))))
    
sched.start()

last ran: 2017-05-13 15:11:17.984329-04:00
last ran: 2017-05-13 15:11:37.739138-04:00
last ran: 2017-05-13 15:11:58.162087-04:00
last ran: 2017-05-13 15:12:17.752070-04:00
last ran: 2017-05-13 15:12:37.674864-04:00
last ran: 2017-05-13 15:12:57.540750-04:00
last ran: 2017-05-13 15:13:17.632709-04:00
last ran: 2017-05-13 15:13:38.189650-04:00
last ran: 2017-05-13 15:13:58.681627-04:00
last ran: 2017-05-13 15:14:17.867855-04:00
last ran: 2017-05-13 15:14:38.495511-04:00
last ran: 2017-05-13 15:14:58.100693-04:00
last ran: 2017-05-13 15:15:18.205634-04:00
last ran: 2017-05-13 15:15:37.738026-04:00
last ran: 2017-05-13 15:15:58.303723-04:00
last ran: 2017-05-13 15:16:18.171499-04:00
last ran: 2017-05-13 15:16:37.847706-04:00
last ran: 2017-05-13 15:16:57.651588-04:00
last ran: 2017-05-13 15:17:17.930061-04:00
last ran: 2017-05-13 15:17:37.550355-04:00
last ran: 2017-05-13 15:17:57.788645-04:00
last ran: 2017-05-13 15:18:17.498276-04:00
last ran: 2017-05-13 15:18:37.608053-04:00
last ran: 2

Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 04:16:56 UTC)" skipped: maximum number of running instances reached (1)
Job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 04:17:56 UTC)" raised an exception
Traceback (most recent call last):
  File "/opt/conda/lib/python3.5/site-packages/requests/packages/urllib3/response.py", line 232, in _error_catcher
    yield
  File "/opt/conda/lib/python3.5/site-packages/requests/packages/urllib3/response.py", line 505, in read_chunked
    chunk = self._handle_chunk(amt)
  File "/opt/conda/lib/python3.5/site-packages/requests/packages/urllib3/response.py", line 474, in _handle_chunk
    returned_chunk = self._fp._safe_read(self.chunk_left)
  File "/opt/conda/lib/python3.5/http/client.py", line 607, in _safe_read
    chunk = self.fp.read(min(amt, MAXAMOUNT))
  File "/opt/conda/lib/python3.5/socket.py", line 575, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handlin

last ran: 2017-05-14 14:17:56.910751-04:00
last ran: 2017-05-14 14:18:18.092460-04:00
last ran: 2017-05-14 14:18:37.871157-04:00
last ran: 2017-05-14 14:18:57.854092-04:00
last ran: 2017-05-14 14:19:18.145119-04:00
last ran: 2017-05-14 14:19:38.431827-04:00
last ran: 2017-05-14 14:19:58.160009-04:00
last ran: 2017-05-14 14:20:18.718829-04:00
last ran: 2017-05-14 14:20:38.515293-04:00
last ran: 2017-05-14 14:20:58.268406-04:00
last ran: 2017-05-14 14:21:18.042979-04:00
last ran: 2017-05-14 14:21:37.913077-04:00
last ran: 2017-05-14 14:21:58.073319-04:00
last ran: 2017-05-14 14:22:18.184291-04:00
last ran: 2017-05-14 14:22:38.038491-04:00
last ran: 2017-05-14 14:23:00.781788-04:00
last ran: 2017-05-14 14:23:17.917895-04:00
last ran: 2017-05-14 14:23:38.202090-04:00
last ran: 2017-05-14 14:23:58.244171-04:00
last ran: 2017-05-14 14:24:17.741253-04:00
last ran: 2017-05-14 14:24:37.814760-04:00
last ran: 2017-05-14 14:24:57.752415-04:00
last ran: 2017-05-14 14:25:18.107566-04:00
last ran: 2

Run time of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 22:06:16 UTC)" was missed by 0:00:17.715984
Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 22:06:36 UTC)" skipped: maximum number of running instances reached (1)
Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 22:06:56 UTC)" skipped: maximum number of running instances reached (1)
Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 23:07:36 UTC)" skipped: maximum number of running instances reached (1)
Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 23:48:16 UTC)" skipped: maximum number of running instances reached (1)
Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 23:57:36 UTC)" skipped: maximum number of running instances reached (1)
Execution of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-14 23:57:56 UTC)" skipped: maximu

last ran: 2017-05-14 19:54:54.345080-04:00
last ran: 2017-05-14 20:02:58.225580-04:00
last ran: 2017-05-14 20:03:17.712939-04:00
last ran: 2017-05-14 20:03:45.418115-04:00
last ran: 2017-05-14 20:04:02.746559-04:00
last ran: 2017-05-14 20:04:23.399254-04:00
last ran: 2017-05-14 20:04:39.181871-04:00
last ran: 2017-05-14 20:04:59.766331-04:00
last ran: 2017-05-14 20:05:29.674025-04:00
last ran: 2017-05-14 20:05:41.852011-04:00
last ran: 2017-05-14 20:06:00.491872-04:00
last ran: 2017-05-14 20:06:20.179036-04:00
last ran: 2017-05-14 20:06:46.062151-04:00
last ran: 2017-05-14 20:07:02.040925-04:00
last ran: 2017-05-14 20:07:25.827238-04:00
last ran: 2017-05-14 20:07:42.170539-04:00
last ran: 2017-05-14 20:08:04.198895-04:00
last ran: 2017-05-14 20:08:31.629734-04:00


...

last ran: 2017-05-14 22:55:45.082627-04:00
last ran: 2017-05-14 22:55:58.583695-04:00
last ran: 2017-05-14 22:56:17.902303-04:00
last ran: 2017-05-14 22:56:38.015490-04:00
last ran: 2017-05-14 22:56:58.617812-04:00
last ran: 2017-05-14 22:57:18.121207-04:00
last ran: 2017-05-14 22:57:39.492725-04:00
last ran: 2017-05-14 22:57:58.136708-04:00
last ran: 2017-05-14 22:58:18.174037-04:00
last ran: 2017-05-14 22:58:38.034030-04:00
last ran: 2017-05-14 22:59:00.853177-04:00
last ran: 2017-05-14 22:59:18.128325-04:00
last ran: 2017-05-14 22:59:38.081538-04:00
last ran: 2017-05-14 22:59:59.565238-04:00
last ran: 2017-05-14 23:00:17.847594-04:00
last ran: 2017-05-14 23:00:38.010552-04:00
last ran: 2017-05-14 23:00:58.213718-04:00
last ran: 2017-05-14 23:01:18.319118-04:00
last ran: 2017-05-14 23:01:38.880948-04:00
last ran: 2017-05-14 23:01:58.021559-04:00
last ran: 2017-05-14 23:02:17.679151-04:00
last ran: 2017-05-14 23:02:37.841837-04:00
last ran: 2017-05-14 23:02:58.055340-04:00
last ran: 2

Run time of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-15 04:23:36 UTC)" was missed by 0:00:02.503330


last ran: 2017-05-15 00:23:37.727576-04:00
last ran: 2017-05-15 00:23:57.990345-04:00
last ran: 2017-05-15 00:24:20.325554-04:00
last ran: 2017-05-15 00:24:38.106112-04:00
last ran: 2017-05-15 00:24:58.349709-04:00
last ran: 2017-05-15 00:25:18.142714-04:00
last ran: 2017-05-15 00:25:38.398869-04:00
last ran: 2017-05-15 00:25:59.573071-04:00
last ran: 2017-05-15 00:26:18.189336-04:00
last ran: 2017-05-15 00:26:37.908136-04:00
last ran: 2017-05-15 00:26:58.219645-04:00
last ran: 2017-05-15 00:27:18.493853-04:00
last ran: 2017-05-15 00:27:38.020460-04:00
last ran: 2017-05-15 00:27:57.834175-04:00
last ran: 2017-05-15 00:28:18.006952-04:00
last ran: 2017-05-15 00:28:38.149274-04:00
last ran: 2017-05-15 00:28:58.194312-04:00
last ran: 2017-05-15 00:29:17.981512-04:00
last ran: 2017-05-15 00:29:38.075391-04:00
last ran: 2017-05-15 00:29:58.174622-04:00
last ran: 2017-05-15 00:30:17.969929-04:00
last ran: 2017-05-15 00:30:37.841600-04:00
last ran: 2017-05-15 00:30:57.990880-04:00
last ran: 2

Run time of job "timed_job (trigger: interval[0:00:20], next run at: 2017-05-15 05:32:16 UTC)" was missed by 0:00:15.962449


last ran: 2017-05-15 01:32:27.996425-04:00
last ran: 2017-05-15 01:32:37.671865-04:00
last ran: 2017-05-15 01:32:57.754865-04:00
last ran: 2017-05-15 01:33:17.816803-04:00
last ran: 2017-05-15 01:33:37.657987-04:00
last ran: 2017-05-15 01:33:57.529147-04:00
last ran: 2017-05-15 01:34:17.582946-04:00
last ran: 2017-05-15 01:34:37.710180-04:00
last ran: 2017-05-15 01:34:57.446808-04:00
last ran: 2017-05-15 01:35:17.880770-04:00
last ran: 2017-05-15 01:35:37.378946-04:00
last ran: 2017-05-15 01:35:58.040099-04:00
last ran: 2017-05-15 01:36:17.882938-04:00
last ran: 2017-05-15 01:36:37.315083-04:00
last ran: 2017-05-15 01:36:57.916122-04:00
last ran: 2017-05-15 01:37:17.647633-04:00
last ran: 2017-05-15 01:37:37.805104-04:00
last ran: 2017-05-15 01:37:57.675102-04:00
last ran: 2017-05-15 01:38:17.557116-04:00
last ran: 2017-05-15 01:38:37.193633-04:00
last ran: 2017-05-15 01:38:58.606500-04:00
last ran: 2017-05-15 01:39:17.864990-04:00
last ran: 2017-05-15 01:39:37.791020-04:00
last ran: 2

In [7]:
#print('last ran: ' + str(datetime.now()) + '\ncount : ' + str(i) + '\nnew find : ' + str(newfind))

In [8]:
#!pip install awscli

In [100]:
#request to ebay
api_request = {
            #'keywords': u'niño',
            'keywords': 'Macbook Pro Retina 16gb 15',
            'sortOrder': 'StartTimeNewest'}

try:
    api = Connection(appid="GeoffCou-macsearc-PRD-4090b840d-a41c0a0b", config_file=None)
    response = api.execute('findItemsAdvanced', api_request)
except ConnectionError as e:
    print(e)
    
r = response.dict()

In [109]:
r

{'ack': 'Success',
 'itemSearchURL': 'http://www.ebay.com/sch/i.html?_nkw=Macbook+Pro+Retina+16gb+15&_ddo=1&_ipg=100&_pgn=1&_sop=10',
 'paginationOutput': {'entriesPerPage': '100',
  'pageNumber': '1',
  'totalEntries': '584',
  'totalPages': '6'},
 'searchResult': {'_count': '100',
  'item': [{'autoPay': 'false',
    'condition': {'conditionDisplayName': 'Used', 'conditionId': '3000'},
    'country': 'US',
    'galleryURL': 'http://thumbs2.ebaystatic.com/m/m0gruVlWRFwFFZd23Yo5Zyg/140.jpg',
    'globalId': 'EBAY-US',
    'isMultiVariationListing': 'false',
    'itemId': '112404133605',
    'listingInfo': {'bestOfferEnabled': 'false',
     'buyItNowAvailable': 'false',
     'endTime': '2017-05-12T20:11:15.000Z',
     'gift': 'false',
     'listingType': 'Auction',
     'startTime': '2017-05-11T20:12:15.000Z'},
    'location': 'Emeryville,CA,USA',
    'paymentMethod': ['PayPal', 'VisaMC'],
    'postalCode': '94608',
    'primaryCategory': {'categoryId': '111422',
     'categoryName': 'Ap

In [104]:
#create dict to import into pandas
dates = {}
for i in range(100):
    dates[i] = {}
    dates[i]['d'] = r['searchResult']['item'][i]['listingInfo']['startTime']
    dates[i]['end'] = r['searchResult']['item'][i]['listingInfo']['endTime']
    dates[i]['buy'] = r['searchResult']['item'][i]['listingInfo']['listingType']
    dates[i]['currency'] = r['searchResult']['item'][i]['sellingStatus']['currentPrice']['_currencyId']
    dates[i]['price'] = r['searchResult']['item'][i]['sellingStatus']['currentPrice']['value']
    dates[i]['return'] = r['searchResult']['item'][i]['returnsAccepted']
    dates[i]['condition'] = r['searchResult']['item'][i]['condition']['conditionId']
    dates[i]['url'] = r['searchResult']['item'][i]['viewItemURL']
    #dates[i]['categoryid'] = r['searchResult']['item'][i]['primaryCategory']['categoryId']
    #dates[i]['shiping'] = r['searchResult']['item'][i]['shippingInfo']['shippingType']
    dates[i]['itemid'] = r['searchResult']['item'][i]['itemId']

#dict df
d = pd.DataFrame(dates)
d = d.T

In [108]:
d.head(3)

,buy,condition,currency,d,end,itemid,price,return,url
0,Auction,3000,USD,2017-05-11T20:12:15.000Z,2017-05-12T20:11:15.000Z,112404133605,1.0,true,http://www.ebay.com/itm/2013-Apple-MacBook-Pro-Retina-15-i7-2-8Ghz-16GB-768GB-SSD-Grade-B-see-notes-/112404133605
1,StoreInventory,3000,USD,2017-05-11T19:59:37.000Z,2017-05-18T19:59:37.000Z,292114144144,1539.0,true,http://www.ebay.com/itm/15-Apple-MacBook-Pro-Retina-2013-2-6GHz-Quad-Core-i7-16GB-RAM-512GB-ME874LL-A-/292114144144
2,FixedPrice,3000,USD,2017-05-11T19:56:06.000Z,2017-06-10T19:56:06.000Z,222508140870,2199.0,false,http://www.ebay.com/itm/MacBook-Pro-15-Retina-I7-2-5ghz-1TB-SSD-16GB-2015-Office-Cs6-Apple-Care-2018-/222508140870


In [ ]:
#convert time
d['d'] = d['d'].str.replace('T',' ')
d['d'] = d['d'].str.replace('Z','')
d['d'] = pd.to_datetime(d['d'])
d['end'] = d['end'].str.replace('T',' ')
d['end'] = d['end'].str.replace('Z','')
d['end'] = pd.to_datetime(d['end'])

#to numeric
for col in d.columns:
    if col not in ['time','d','end']:
        try:
            d[col] = pd.to_numeric(d[col])
        except:
            ValueError

#dont cut the 
pd.set_option('display.max_colwidth', -1)

#time fudging
d['diff'] = (d['end'] - datetime.now()).astype('timedelta64[m]')
#d['d'] = d['d'].apply(timezone('US/Eastern').localize)
#d['diff'] = d['end'] - datetime.now(timezone('US/Eastern'))

In [ ]:
accountSID = 'ACba8--------------------12eb8'
authToken = 'fbe13c9-----------------3b03b6'
client = Client(accountSID, authToken)
myTwilioNumber = '+13-----------7'
myCellPhone = '+1----------2'

In [ ]:
accountSID = 'ACba8d171da71289807d6aa640b2e12eb8'
authToken = 'fbe13c9f5a79c3b4922530d0e93b03b6'
client = Client(accountSID, authToken)
myTwilioNumber = '+13399707227'
myCellPhone = '+13392239032'

In [ ]:
#df mask for the data
price = 750
window = 600

#buyitnow mask
mask = ((d['buy'] == 'FixedPrice') 
        & (d['categoryid'] == 111422) 
        & (d['price'] <= price))

#auction mask
# mask = ((d['buy'] == 'Auction') 
#         & (d['diff'] < window) 
#         & (d['categoryid'] == 111422) 
#         & (d['price'] <= price))

In [92]:
#initialize list
#itemid_list = []

#loop over dictionary
url = ''
for i in range(len(d[mask])):
    if d[mask]['itemid'].iloc[i] not in itemid_list:
        itemid_list.append(d[mask]['itemid'].iloc[i])
        #print(i)
        url = str(d[mask]['url'].iloc[i]) + ' AND ' + url 
        print(url)
        #message = client.messages.create(body=str(url),from_=myTwilioNumber, to=myCellPhone)
#print(d[mask])
#print every run
print('last ran: ' + str(datetime.now(timezone('US/Eastern'))))

http://www.ebay.com/itm/Early-2013-15-Macbook-Pro-Retina-16GB-RAM-512-SSD-/252929543003 AND 
last ran: 2017-05-10 13:32:28.045511-04:00


In [110]:
d[mask]['url']

25    http://www.ebay.com/itm/15-RETINA-Apple-MacBook-Pro-i7-2-5-1-TB-SSD-16GB-RAM-APPLECARE-2018-/122490441679             
45    http://www.ebay.com/itm/Apple-MacBook-Pro-15-Quad-Core-i7-Pre-Retina-Mac-OS-2016-16GB-RAM-1TB-SSD-HYBRID-/272666699362
Name: url, dtype: object

In [70]:
pd.Timedelta(d['end'] - d['d'])#.total_seconds()

ValueError: Value must be Timedelta, string, integer, float, timedelta or convertible

In [85]:
d['diff'] = (d['end'] - datetime.now()).astype('timedelta64[m]')

In [93]:
d.sort_values(by='diff')

,buy,categoryid,condition,currency,d,end,itemid,price,return,shiping,url,diff
45,Auction,111422,3000,USD,2017-05-09 17:40:14,2017-05-10 17:39:15,222505469576,830.00,false,Flat,http://www.ebay.com/itm/MacBook-Pro-Retina-15-inch-Late-2013-16gb-RAM-i7-2-6GHZ-1TB-Flash-Storage-/222505469576,6.0
25,Auction,111422,3000,USD,2017-05-09 23:46:38,2017-05-10 23:46:38,252929543003,380.00,false,Free,http://www.ebay.com/itm/Early-2013-15-Macbook-Pro-Retina-16GB-RAM-512-SSD-/252929543003,374.0
23,AuctionWithBIN,111422,3000,USD,2017-05-10 00:40:03,2017-05-11 00:40:03,152543245664,1200.00,false,Free,http://www.ebay.com/itm/15-Apple-MacBook-Pro-Retina-Mid-2015-2-2Ghz-i7-16GB-256GB-SSD-Original-Box-/152543245664,427.0
92,StoreInventory,111422,3000,USD,2017-05-08 14:23:32,2017-05-11 14:23:32,142376769148,849.95,true,Calculated,http://www.ebay.com/itm/Apple-Macbook-Pro-Retina-15-2012-Laptop-2-6-GHz-i7-16-GB-512-GB-MC976LL-A-/142376769148,1251.0
90,Auction,111422,3000,USD,2017-05-08 15:32:01,2017-05-11 15:32:01,142376832466,950.00,false,Calculated,http://www.ebay.com/itm/Apple-MacBook-Pro-15-Retina-A1398-Late-2013-2-3GHz-Core-i7-16GB-256GB-Sierra-/142376832466,1319.0
88,Auction,111422,3000,USD,2017-05-08 16:10:58,2017-05-11 16:10:58,262974206351,730.00,true,Free,http://www.ebay.com/itm/Apple-MacBook-Pro-RETINA-15-Laptop-2013-2-7-GHz-i7-16GB-RAM-512GB-SSD-/262974206351,1358.0
77,StoreInventory,111422,3000,USD,2017-05-08 23:10:15,2017-05-11 23:10:15,112400925170,1499.99,true,Free,http://www.ebay.com/itm/2015-Apple-MacBook-Pro-Retina-15-Laptop-256GB-2-2GHz-16GB-RAM-A1398-MJLQ2LL-A-/112400925170,1777.0
76,Auction,111422,3000,USD,2017-05-08 23:57:17,2017-05-11 23:56:17,382076232989,906.00,true,Free,http://www.ebay.com/itm/Apple-Macbook-Pro-15-Retina-ME665LL-A-Core-i7-2-7Ghz-16GB-Ram-512GB-SSD-/382076232989,1823.0
75,Auction,111422,3000,USD,2017-05-09 00:05:18,2017-05-12 00:04:17,401323781373,899.99,true,Free,http://www.ebay.com/itm/Apple-Macbook-Pro-Retina-15-ME294LL-A-Core-i7-2-3Ghz-16GB-Ram-500GB-SSD-/401323781373,1831.0
74,Auction,111422,3000,USD,2017-05-09 00:13:52,2017-05-12 00:13:52,152541992167,1269.00,false,Calculated,http://www.ebay.com/itm/Apple-15-RETINA-MacBook-Pro-2-5Ghz-i7-16GB-RAM-512GB-SSD-MJLT2LL-A-/152541992167,1841.0
